## Ejercicio attrition

1. Carga el dataset [attrition de `kaggle`](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset?resource=download) (Hay que loguearse en kaggle y descargar el csv)
   1. Clasificacion de si el empleado deja o no la empresa (el target es la columna Attrition).
   2. Usaremos TODAS las columnas disponibles del dataset. No hace falta volverse loco con el feature engineering, pero es posible que alguna cosa haya que hacer para poder meter todas las columnas. 
   
2. Prueba todos los métodos de clasificación vistos hasta ahora mediante GridSearchCV (logistic regression, SVM, al menos un metodo de boosting, random forest y MLP). Obligatorio usarlos todos por separado usando pipelines. Se puede hacer todo en sklearn.
   1. Hay que reportar la performance de los modelos con al menos: Accuracy, precission, recall, F1-Score y AUC-ROC
   2. IMPORTANTE: Comentar cual es el mejor metodo y cual es la mejor metrica para este caso de uso concreto.
   
3. Utiliza uno de los métodos para sacar la importancia de las variables del modelo escogido.
   **¿Qué es?**
   Serie de técnicas que asignan puntuaciones a las variables independientes de un modelo predictivo en función de su importancia relativa al realizar una predicción sobre la variable dependiente o target. 
   IMPORTANTE: Antes de obtener el feature importance siempre hay que evaluar la capacidad predictiva del modelo implementado.
   **Tipos**
   1. Métodos "built in" en modelos intrinsecamente intepretables (ya vistos al estudiar los modelos lineales y los modelos basados en árboles de decision). 
   2. Permutation importance
   3. Drop columns importance
   
4. Utilizar un ultimo metodo que sea un stacking (ensemble) de al menos 4 de los metodos anteriores. Obtener las metricas anteriores (acc, prec, recall, etc..)
   
5. Valoracion y comentarios de los resultados obtenidos al final. 

In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [4]:
colum =[]
colum_num=[[]]
for col in df.columns:
    if df[col].dtype == object:
        colum_num.append(col)

In [6]:
df = pd.DataFrame(colum_num)

In [ ]:
X = df
y = df["Attrition"]

In [95]:
#X_train, X_test, y_train, y_test = train_test_split(df.drop("Attrition", axis=1), df["Attrition"], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

ValueError: Found input variables with inconsistent numbers of samples: [1, 1470]

In [ ]:
pipe_rf = Pipeline(steps = [
    ("mms", MinMaxScaler()),
    ("rf", RandomForestClassifier())
])
rfc_param = {
    "rf__n_estimators" : [100,50,200],
    "rf__max_depth":[3,5,10]
}

clf_pipe = GridSearchCV(estimator= pipe_rf,
                        param_grid = rfc_param,
                        scoring="balanced_accuracy",
                        n_jobs=-1,
                        cv=5)

clf_pipe.fit(X_train,y_train)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3472152703.py, line 12)

In [80]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, auc, recall_score

In [ ]:
predictions = clf_pipe.best_estimator_.predict(X_test)

In [ ]:
print(accuracy_score(y_test, predictions))
print(f1_score(y_test, predictions))
print(precision_score(y_test, predictions))
print(auc(y_test, predictions))
print(recall_score(y_test, predictions))

In [ ]:
(y_test=="Yes").astype(int)

In [ ]:
X = df[['Age', 'BusinessTravel', 'DailyRate', 'Department',
        'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
        'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
        'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
        'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
        'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
        'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
        'YearsWithCurrManager']]
y = df["Attrition"]

In [4]:
le = LabelEncoder()
df_num = df.copy()

In [18]:
# df_num = pd.get_dummies(df_num, columns=['Attrition_Yes'], prefix='Attrition_Yes', drop_first=True)
# df_num = pd.get_dummies(df_num, columns=['Attrition_Yes_True'], prefix='Attrition_Yes_True', drop_first=True)

In [13]:
df_num["BusinessTravel"] = le.fit_transform(df_num['BusinessTravel'])
df_num["MaritalStatus"] = le.fit_transform(df_num['MaritalStatus'])
df_num["EducationField"] = le.fit_transform(df_num['EducationField'])
df_num["Department"] = le.fit_transform(df_num['Department'])
df_num["JobRole"] = le.fit_transform(df_num['JobRole'])
df_num["Over18"] = le.fit_transform(df_num['Over18'])
df_num["OverTime"] = le.fit_transform(df_num['OverTime'])

In [48]:
# df.describe()

In [49]:
# df_num.info()

In [35]:
X_num = df_num[['Age', 'BusinessTravel', 'DailyRate', 'Department',
        'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
        'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
        'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
        'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
        'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
        'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
        'YearsWithCurrManager']]
y_num = df_num["Attrition"]

In [36]:
X = df_num[['Age', 'BusinessTravel', 'DailyRate', 'Department',
        'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
        'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
        'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
        'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
        'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
        'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
        'YearsWithCurrManager']]
y = df_num["Attrition"]

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_num,y_num, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1176, 34)
(294, 34)
(1176,)
(294,)


In [51]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
X_train_processed = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
# X_test_processed = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True) # ¡Importante para X_test también!

In [54]:
rfc = Pipeline(steps = [
    ("le", LabelEncoder()),
    ("rfc", RandomForestClassifier())
])
rfc_param = {
    "rfc__n_estimators" : [100,50,200],
    "rfc__max_depth":[3,5,10],
    "rfc__min_samples_split":[2,3,4,5],
    "rfc__min_samples_leaf":[1,2,3,4],
}

clf_pipe = GridSearchCV(estimator= rfc,
                        param_grid = rfc_param,
                        n_jobs=-1,
                        cv=5)

clf_pipe.fit(X_train,y_train)

ValueError: 
All the 720 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\maxi\miniconda3\envs\data_analysis_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\maxi\miniconda3\envs\data_analysis_env\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\maxi\miniconda3\envs\data_analysis_env\Lib\site-packages\sklearn\pipeline.py", line 654, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

In [ ]:
# error_score='raise'

In [ ]:
parameters = 


clf = GridSearchCV(estimator = rfc,
                    param_grid = parameters,
                    n_jobs = -1,
                    cv = 10)

clf.fit(X_train, y_train)

NameError: name 'rfc' is not defined